In [ ]:
# ────────────────────────────────────────────────────────────────
# 02_change_point_model.ipynb
# Simple SINGLE change point model with PyMC
# ────────────────────────────────────────────────────────────────

import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

RANDOM_SEED = 20232023
rng = np.random.default_rng(RANDOM_SEED)

# ─── Load prepared data ─────────────────────────────────────────────
df = pd.read_csv("../data/processed/brent_model_ready.csv", parse_dates=["Date"])
df = df.dropna(subset=["price"])

y = df["price"].values          # modeling prices directly (you can switch to log_price)
N = len(y)
t = np.arange(N)

print(f"Modeling {N} days of data")


with pm.Model() as model_single:

    # Change point (day index)
    tau = pm.DiscreteUniform("tau", lower=10, upper=N-10)


    mu_1 = pm.Normal("mu_1", mu=70, sigma=30)
    mu_2 = pm.Normal("mu_2", mu=70, sigma=30)

    # Common noise (you can make two sigmas later)
    sigma = pm.HalfNormal("sigma", sigma=15)

    # Switch mean
    mu = pm.math.switch(t < tau, mu_1, mu_2)

    # Likelihood
    obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=y)

    # Sample!
    trace = pm.sample(
        draws=2000,
        tune=2000,
        chains=4,
        target_accept=0.92,
        random_seed=RANDOM_SEED,
        return_inferencedata=True
    )

# ─── Diagnostics ────────────────────────────────────────────────────
print(az.summary(trace, var_names=["tau", "mu_1", "mu_2", "sigma"]))

az.plot_trace(trace, var_names=["tau", "mu_1", "mu_2", "sigma"])
plt.tight_layout()
plt.show()

az.plot_posterior(trace, var_names=["tau", "mu_1", "mu_2"])
plt.show()

# ─── Plot posterior change point probability ────────────────────────
tau_samples = trace.posterior["tau"].values.flatten()

plt.figure(figsize=(12,5))
plt.hist(tau_samples, bins=np.arange(0, N+1, 5), density=True, color="teal", alpha=0.7)
plt.title("Posterior probability of change point location")
plt.xlabel("Day index (0 = start of dataset)")
plt.ylabel("Posterior density")


xticks_idx = np.linspace(0, N, 8, dtype=int)
xtick_dates = df["Date"].iloc[xticks_idx].dt.strftime("%Y-%m")
plt.xticks(xticks_idx, xtick_dates, rotation=30)

plt.tight_layout()
plt.show()

# Most probable change point
map_tau = int(az.summary(trace)["mean"]["tau"])
print(f"MAP change point day index: {map_tau}")
print("Date:", df["Date"].iloc[map_tau])

c:\Users\Hp\Downloads\Tenx_academy\brent-oil-change-point-analysis\.venv\Lib\site-packages\arviz\__init__.py:50: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


Modeling 2729 days of data

You can find the C code in this temporary file: C:\Users\Hp\AppData\Local\Temp\pytensor_compilation_error_prm3pg9b


CompileError: Compilation failed (return status=1):
"C:\msys64\ucrt64\bin\g++.EXE" -shared -g -Wno-c++11-narrowing -fno-exceptions -fno-unwind-tables -fno-asynchronous-unwind-tables -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -m64 -DMS_WIN64 -I"c:\Users\Hp\Downloads\Tenx_academy\brent-oil-change-point-analysis\.venv\Lib\site-packages\numpy\_core\include" -I"C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.3312.0_x64__qbz5n2kfra8p0\include" -I"c:\Users\Hp\Downloads\Tenx_academy\brent-oil-change-point-analysis\.venv\Lib\site-packages\pytensor\link\c\c_code" -L"C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.3312.0_x64__qbz5n2kfra8p0\libs" -L"C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.3312.0_x64__qbz5n2kfra8p0" -o "C:\Users\Hp\AppData\Local\PyTensor\compiledir_Windows-11-10.0.26200-SP0-Intel64_Family_6_Model_126_Stepping_5_GenuineIntel-3.13.12-64\lazylinker_ext\lazylinker_ext.pyd" "C:\Users\Hp\AppData\Local\PyTensor\

In [2]:
# Variant: different variance before / after
with pm.Model() as model_diff_sigma:
    tau = pm.DiscreteUniform("tau", lower=100, upper=N-100)
    
    mu1 = pm.Normal("mu1", 70, 35)
    mu2 = pm.Normal("mu2", 70, 35)
    
    sigma1 = pm.HalfNormal("sigma1", sigma=20)
    sigma2 = pm.HalfNormal("sigma2", sigma=20)
    
    sigma_t = pm.math.switch(t < tau, sigma1, sigma2)
    mu_t   = pm.math.switch(t < tau, mu1,   mu2)
    
    obs = pm.Normal("obs", mu=mu_t, sigma=sigma_t, observed=y)
    
    trace_diff = pm.sample(1000, tune=1000, chains=4, target_accept=0.93)

trace_diff.to_netcdf("../traces/diff_sigma_trace.nc")


You can find the C code in this temporary file: C:\Users\Hp\AppData\Local\Temp\pytensor_compilation_error_ucdx62lv


CompileError: Compilation failed (return status=1):
"C:\msys64\ucrt64\bin\g++.EXE" -shared -g -Wno-c++11-narrowing -fno-exceptions -fno-unwind-tables -fno-asynchronous-unwind-tables -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -m64 -DMS_WIN64 -I"c:\Users\Hp\Downloads\Tenx_academy\brent-oil-change-point-analysis\.venv\Lib\site-packages\numpy\_core\include" -I"C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.3312.0_x64__qbz5n2kfra8p0\include" -I"c:\Users\Hp\Downloads\Tenx_academy\brent-oil-change-point-analysis\.venv\Lib\site-packages\pytensor\link\c\c_code" -L"C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.3312.0_x64__qbz5n2kfra8p0\libs" -L"C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.3312.0_x64__qbz5n2kfra8p0" -o "C:\Users\Hp\AppData\Local\PyTensor\compiledir_Windows-11-10.0.26200-SP0-Intel64_Family_6_Model_126_Stepping_5_GenuineIntel-3.13.12-64\lazylinker_ext\lazylinker_ext.pyd" "C:\Users\Hp\AppData\Local\PyTensor\